<a href="https://colab.research.google.com/github/canisius-DAT512/Spring-23-Week-8/blob/main/Ecommerce_and_Reccomendation_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E-Commerce Analysis on Google Cloud
DAT 512 Canisius College <br>
Adjunct Professor Paul Lambson

Learning Objectives
- Walk through query building for analysis
- Find key assumptions for The Hub dataset
- Perform an analysis on business, product and basket
- Build a reccomendation engine


# Query Building for Analysis
When presented with a problem for analysis we can take on an a preperatory step to simplify the process.

**Psuedo-code** `In computer science, pseudocode is a plain language description of the steps in an algorithm or another system.`

Breaking down a problem into the component pieces, understanding the data sources, flow and logic gates at the onset can direct make the process more effiecient.

What parts of a problem need to be broken down?
1.   Understand the question, understand why it's being asked, understand how the answer will be used.
2.   Identify the data elements needed to answer the question
3.   Determine the method of getting all the data elements on the same environment
4.   Evaluate inclusion and/or exclusion criteria
5.   Define the flow of the data and calcualtion that should happen at each step

What does psuedocode look like?

Scratch paper, non-operable code, diagrams, whatever medium allows for interegation of steps prior to compilation.

## Example psuedocode
Given the question, what is the highest selling product.

- Will need products information and sales information
- Need to join the tables on a product column
- Need all valid sales data
- Exclude sales that aren't valid, look at all `status` values

<table>
<tr>
<th>Psuedocode</th>
<th>Standard SQL</th>
</tr>
<tr>
<td>
  
```
select most of the columns
from the orders table and the product table
join them on product id
limit to only valid sales (no returns, etc.)
```
  
</td>
<td>

```sql
SELECT *
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
  `bigquery-public-data.thelook_ecommerce.products` p
ON
  oi.product_id = p.id
WHERE status NOT IN ('Returned','Cancelled')
```

</td>
</tr>
</table>



In [ ]:
# authenticate with the google account used in console
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pandas as pd

project_id = 'tla-consultants'

df = pd.io.gbq.read_gbq(f'''
''',project_id=project_id)

# Business Analysis
- Count of sales by month over time
- Sales over time
- Sales by status
- Orders fulfilled by distribution center

# Product Analysis
- What are the category sales distribution
- What are the highest selling products?
- What are the worst selling products?
- Does it differ by year?
- What are the most returned items?
- Does price point drive sales?


# Basket Analysis
- What is the average baasket size?
- What is the makeup of shopping carts?
- What sells well together?

# Recomendation Engine


In [ ]:
# Create a dataframe with only users and products
sql = '''
SELECT
  user_id, product_id
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
  `bigquery-public-data.thelook_ecommerce.products` p
ON
  oi.product_id = p.id
WHERE status NOT IN ('Returned','Cancelled')
'''

import pandas as pd

project_id = 'tla-consultants'

df = pd.io.gbq.read_gbq(sql,project_id=project_id)
df.shape

In [ ]:
# make a dataframe with only product information
sql = '''
SELECT
  *
FROM
`bigquery-public-data.thelook_ecommerce.products`
'''

import pandas as pd

project_id = 'tla-consultants'

products = pd.io.gbq.read_gbq(sql,project_id=project_id)
products.shape

In [ ]:
# downselect to products that have sold at least 8 times
product_freq = df['product_id'].value_counts()
top_products = product_freq[product_freq>8].index
print(top_products.shape)
df = df[df['product_id'].isin(top_products)]
df.shape

In [ ]:
# downselect to users who have purchased more than 1 item
product_users = df.groupby('user_id')['product_id'].nunique()
users_w_products = product_users[product_users>1].index

df = df[df['user_id'].isin(users_w_products)]
df.shape

In [ ]:
# transpose to dummies
product_dummy = pd.get_dummies(df['product_id'])
# join in user_id to dummies
up_dummy = product_dummy.join(df['user_id'])
up_dummy.shape

In [ ]:
up_dummy.head()

In [ ]:
# group by user ids get max, so binary flag
up_grouped = up_dummy.groupby('user_id').max()

# create corr matrix
corr_matrix = up_grouped.corr()
corrs = corr_matrix.unstack().reset_index()
corrs.columns = ['Product_A','Product_B','Corr_Value']

# add product information to corr values
corrs_names = corrs.merge(products[['id','name']], left_on='Product_A', right_on='id').merge(products[['id','name']], left_on='Product_B', right_on='id')


In [ ]:
corrs_names.head()

In [ ]:
# Let's look for products with strong correlation values with other products 
corrs_names[corrs_names['Corr_Value']>0]['Product_A'].value_counts()

In [ ]:
corrs_names[corrs_names['Product_A']==22810].sort_values(by='Corr_Value',ascending=False).head(10)